In [38]:
CSV_PATH = "scraper/output/agences_combined_lokker_ready.csv"  

In [39]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

# === Chargement des données ===
df = pd.read_csv("scraper/output/agences_combined_lokker_ready.csv")
df = df.dropna(subset=['latitude', 'longitude'])
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df = df.dropna(subset=['latitude', 'longitude'])

# === Carte centrale France ===
m = folium.Map(location=[46.6, 2.4], zoom_start=6, tiles="CartoDB positron")

# === JS pour style cluster dynamique ===
icon_create_function = """
function(cluster) {
  var count = cluster.getChildCount();
  var size = 30;
  var color = 'rgba(144, 238, 144, 0.8)';

  if (count < 10) {
    size = 30;
    color = 'rgba(144, 238, 144, 0.8)';
  } else if (count < 50) {
    size = 40;
    color = 'rgba(255, 215, 0, 0.8)';
  } else if (count < 200) {
    size = 50;
    color = 'rgba(255, 140, 0, 0.8)';
  } else {
    size = 60;
    color = 'rgba(220, 20, 60, 0.9)';
  }

  return new L.DivIcon({
    html: '<div style="background:' + color + '; border-radius:50%; width:' + size + 'px; height:' + size + 'px; line-height:' + size + 'px; color:white; text-align:center; font-weight:bold;">' + count + '</div>',
    className: 'marker-cluster',
    iconSize: new L.Point(size, size)
  });
}
"""

# === Regroupement par source (filtrable) ===
sources = df['source'].dropna().unique()

for src in sources:
    fg = folium.FeatureGroup(name=f"Source : {src}")
    cluster = MarkerCluster(icon_create_function=icon_create_function).add_to(fg)

    for _, row in df[df['source'] == src].iterrows():
        popup_html = f"""
        <b>{row['nom']}</b><br>
        {row['adresse']}<br>
        {row['code_postal']}<br>
        <i>Source : {row['source']}</i><br>
        <i>Région : {row['region_source']}</i>
        """
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=6,
            color='#3388ff',
            fill=True,
            fill_color='#3388ff',
            fill_opacity=0.85,
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(cluster)

    fg.add_to(m)

# === Légende interactive ===
folium.LayerControl(collapsed=False).add_to(m)

# === Export final ===
m.save("map_clusters_filtres_points.html")
print("✅ Carte avec filtres + points simples générée.")


✅ Carte avec filtres + points simples générée.


In [40]:
#combien de ligne avec source == 'sg'
#regader le csv pour voir les sources
# === Compter les lignes avec source 'sg' ===
sg_count = df[df['source'] == 'SG'].shape[0]
print(f"Nombre de lignes avec source 'sg' : {sg_count}")
# === Compter les lignes avec source 'bnp' ===
bnp_count = df[df['source'] == 'BNP'].shape[0]
print(f"Nombre de lignes avec source 'bnp' : {bnp_count}")
# === Compter les lignes avec source 'cm' ===
cm_count = df[df['source'] == "Caisse d'Épargne"].shape[0]
print(f"Nombre de lignes avec source 'cm' : {cm_count}")


Nombre de lignes avec source 'sg' : 2210
Nombre de lignes avec source 'bnp' : 1837
Nombre de lignes avec source 'cm' : 3601


4 182 agences bancaires CM
1.500 agences BNP